## Расположение проекта:

In [1]:
proj_path = "C:\\Users\\VG\\Saved Games\\Desktop\\dippractice\\"
proj_path_b = proj_path.replace('\\','/')

## Проверка используемого окружения

In [ ]:
# Текущее окружение: 

import pkg_resources
thisenv = []
for pkg in pkg_resources.Environment().__iter__():
    thisenv.append(pkg + "==" + pkg_resources.get_distribution(pkg).version)

In [ ]:
# Окружение совпадает с requirement-summarization ?

reqenv = []
with open(proj_path + "requirements_summarization.txt") as f:
    lines = f.readlines()
    for l in lines:
        reqenv.append(l.lower().replace('\n',''))
    
print("Лишние: \n")
for pkg in thisenv:
    if pkg not in reqenv:
        print(pkg)
print("-----------")

print("Недостающие: \n")
for pkg in reqenv:
    if pkg not in thisenv:
        print(pkg)
print("-----------")

## Необходимые библиотеки:

In [2]:
import os
import enum
import math
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_text as text
from sentence_transformers import SentenceTransformer
import umap
from sklearn import decomposition

#from sklearn.preprocessing import StandardScaler

## Подготовка датасета
Преобразование исходных танных в виде нескольких таблиц dataframe так, чтобы одно предложение = один сэмпл


In [22]:
dataset_name = "putinspeech"
dataset_path = "data\\politics\\ru\\putinspeech\\"
dataset_path_b = dataset_path.replace('\\','/')

df_commentsinfo = pd.read_pickle(proj_path_b + dataset_path_b + "putinspeech_dataset.pkl")
df_commentsinfo.head(3)

,Unnamed: 0,Nickname,LikedByAuthor,CommentText,Date,LikesCount,isBaseComment,RepliesCount,RepliesID,CommentTextSplitNV
0,0,Реакция на Видео,False,"Кто-то захотел повоевать, а нам просто охота м...",2022-02-23 06:51:52.449724,3200,True,175,"[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3...","[Кто-то захотел повоевать, а нам просто охота ..."
1,1,soiung toiue,False,"🇷🇺🤝🇺🇦 мирного небо, нам братья и сестры. Пусть...",2022-02-27 06:51:52.451693,8,True,0,[],"[мирного небо, нам братья и сестры., Пусть наш..."
2,2,Vlad Romanovsi,False,Братья украинцы мы против войны . То что проис...,2022-02-24 06:51:52.452690,1200,True,179,"[195, 196, 197, 198, 199, 200, 201, 202, 203, ...","[Братья украинцы мы против войны ., То что про..."


In [4]:
class TextToSentenceSplitMethods(enum.Enum):
    Naive = "naive"
    KohenSchroeder = "kohenschroeder"
    
class SplittedSentences:
    def __init__(self, dfsentences, methodd):
        self.sentences = dfsentences
        self.method = methodd

In [23]:
%%time
# Определенные методом Naive предложения:
if "CommentTextSplitNV" in df_commentsinfo.columns:        
    dflist=[]
    for index, row in df_commentsinfo.iterrows():
        for sentence in row["CommentTextSplitNV"]:
            dflist.append(pd.DataFrame([[index,sentence]],columns=["CommentID","CommentSentence"]))

    df_commentstext_NV = pd.concat(dflist,ignore_index=True) 
    commentsNV = SplittedSentences(df_commentstext_NV, TextToSentenceSplitMethods.Naive)
    del df_commentstext_NV

Wall time: 9.65 s


In [ ]:
%%time
# Определенные методом Kohen-Schroeder предложения:
#  - работает только если комментарии на английском языке -
if "CommentTextSplitKS" in df_commentsinfo.columns:        
    dflist=[]
    for index, row in df_commentsinfo.iterrows():
        for sentence in row["CommentTextSplitKS"]:
            dflist.append(pd.DataFrame([[index,sentence]],columns=["CommentID","CommentSentence"]))

    df_commentstext_KS = pd.concat(dflist,ignore_index=True)
    commentsKS = SplittedSentences(df_commentstext_KS, TextToSentenceSplitMethods.KohenSchroeder)
    del df_commentstext_KS

In [24]:
commentsNV.sentences.head(10)

,CommentID,CommentSentence
0,0,"Кто-то захотел повоевать, а нам просто охота м..."
1,0,В войне нет победителей!
2,0,Нет войне и нет гибели человечества!
3,0,Мир во всем мире!
4,0,Все мы Братья!
5,1,"мирного небо, нам братья и сестры."
6,1,"Пусть наши дети незнают, что такое война."
7,2,Братья украинцы мы против войны .
8,2,То что происходит полное безумие !
9,2,Поджигателей и подстрекателей к войне судьба с...


In [25]:
print(commentsNV.sentences.count())
print(commentsKS.sentences.count())

CommentID          28058
CommentSentence    28058
dtype: int64


NameError: name 'commentsKS' is not defined

## Sentence Encoders

In [7]:
class SentenceEncodersTypes(enum.Enum):
    SBERT = "SBERT"
    CMLM = "CMLM"
    LaBSE = "LaBSE"
    
class SentencePreprocessorsTypes(enum.Enum):
    BERT = "BERT"
    CMLM = "CMLM"
    
class SentenceEncoder:
    def __init__(self, encoderr, enctypee, modelnamee):
        self.encoder = encoderr
        self.enctype = enctypee
        self.modelname = modelnamee

class SentencePreprocessor:
    def __init__(self, preprocessorr, preprocessortypee, modelnamee):
        self.preprocessor = preprocessorr
        self.preprocessortype = preprocessortypee
        self.modelname = modelnamee

In [8]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']

SBERT

In [ ]:
ParaphraseMiniLML6v2_encoder = SentenceTransformer(proj_path + "stmodels\\SBERT\\models\\paraphraseMiniLML6v2\\model\\")
ParaphraseMiniLML6v2 = SentenceEncoder(ParaphraseMiniLML6v2_encoder, SentenceEncodersTypes.SBERT, "ParaphraseMiniLML6v2")

tf.constant(ParaphraseMiniLML6v2.encoder.encode(sentences))

In [9]:
from transformers import AutoModel, AutoTokenizer
RuBERT180M_model = AutoModel.from_pretrained(proj_path + "stmodels\\SBERT\\models\\RuBERT180M\\model\\")
RuBERT180M_tokenizer = AutoTokenizer.from_pretrained(proj_path + "stmodels\\SBERT\\models\\RuBERT180M\\tokenizer\\")

class RuBERT180MEncoder :
    def __init__(self, modell, tokenizerr):
        self.model = modell
        self.tokenizer = tokenizerr
        
    def encode(self, sentencess):     
        arrs_to_concat = []

        for sent in sentencess:
            temp = self.tokenizer(sent, return_tensors="pt")
            for key in temp:
                if temp[key].size(dim=1) > 512:
                    temp[key] = temp[key].resize_(1,512)
            embedding = self.model(**temp, output_hidden_states=False).pooler_output
            arrs_to_concat.append(embedding.detach().numpy())
        
        return np.concatenate(arrs_to_concat, axis=0)

RuBERT180M_encoder = RuBERT180MEncoder(RuBERT180M_model,RuBERT180M_tokenizer)
RuBERT180M = SentenceEncoder(RuBERT180M_encoder, SentenceEncodersTypes.SBERT, "RuBERT180M")

tf.constant(RuBERT180M.encoder.encode(sentences))

<tf.Tensor: shape=(3, 768), dtype=float32, numpy=
array([[ 0.04165118, -0.04761205,  0.15364729, ..., -0.18241313,
         0.29493603,  0.11059465],
       [ 0.00193595,  0.03411477,  0.04184175, ..., -0.08403821,
         0.03101917, -0.02019569],
       [-0.08977324,  0.08727846, -0.03098175, ...,  0.05084056,
        -0.02457064, -0.0141814 ]], dtype=float32)>

CMLM (1 GB)

In [10]:
SentEvalV1_encoder = tf.keras.models.load_model(proj_path + "stmodels\\CMLM\\CMLMsenteval\\models\\CMLMsentevalV1\\model\\")
SentEvalV1 = SentenceEncoder(SentEvalV1_encoder, SentenceEncodersTypes.CMLM, "SentEvalV1")

ENuncasedV3_preprocessor = tf.keras.models.load_model(proj_path + "stmodels\\preprocessors\\BERTenUncasedV3\\model\\")
ENuncasedV3 = SentencePreprocessor(ENuncasedV3_preprocessor, SentencePreprocessorsTypes.BERT, "ENuncasedV3")

SentEvalV1.encoder(ENuncasedV3.preprocessor(tf.constant(sentences)))["default"]


Two checkpoint references resolved to different objects (<keras.engine.functional.Functional object at 0x0000018821B7FA20> and <keras.engine.input_layer.InputLayer object at 0x0000018821B7FB38>).

Two checkpoint references resolved to different objects (<keras.engine.functional.Functional object at 0x0000018821B7FA20> and <keras.engine.input_layer.InputLayer object at 0x0000018821B7FB38>).

Two checkpoint references resolved to different objects (<keras.engine.functional.Functional object at 0x0000018821B7FA20> and <keras.engine.input_layer.InputLayer object at 0x0000018821B7FB38>).


<tf.Tensor: shape=(3, 768), dtype=float32, numpy=
array([[ 0.07520977, -0.11222922,  0.005889  , ..., -0.01480528,
        -0.05141396,  0.04096572],
       [ 0.01025412, -0.12978497,  0.0178423 , ..., -0.11460859,
        -0.03255388, -0.01154309],
       [ 0.05583872, -0.09108108, -0.01788832, ...,  0.03635921,
        -0.04670589,  0.08279653]], dtype=float32)>

LaBSE (3 GB)

In [11]:
LaBSEV2_encoder=tf.keras.models.load_model(proj_path + "stmodels\\LaBSE\\models\\LaBSEV2\\model\\")
LaBSEV2 = SentenceEncoder(LaBSEV2_encoder, SentenceEncodersTypes.LaBSE, "LaBSEV2")

MultilingualV2_preprocessor = tf.keras.models.load_model(proj_path + "stmodels\\preprocessors\\CMLMmultilingualV2\\model\\")
MultilingualV2 = SentencePreprocessor(MultilingualV2_preprocessor, SentencePreprocessorsTypes.CMLM, "MultilingualV2")

LaBSEV2.encoder(MultilingualV2.preprocessor(tf.constant(sentences)))["default"]


Two checkpoint references resolved to different objects (<keras.engine.functional.Functional object at 0x00000188EE173D68> and <keras.engine.input_layer.InputLayer object at 0x00000188EE16C860>).

Two checkpoint references resolved to different objects (<keras.engine.functional.Functional object at 0x00000188EE173D68> and <keras.engine.input_layer.InputLayer object at 0x00000188EE16C860>).

Two checkpoint references resolved to different objects (<keras.engine.functional.Functional object at 0x00000188EE173D68> and <keras.engine.input_layer.InputLayer object at 0x00000188EE16C860>).


<tf.Tensor: shape=(3, 768), dtype=float32, numpy=
array([[ 0.03764332, -0.64704597,  0.26479277, ..., -0.16649246,
         0.09929831, -0.06447963],
       [-0.3558307 ,  0.17987226,  0.23107216, ..., -0.00949938,
         0.19046369,  0.15931569],
       [ 0.56712157,  0.49384737,  0.6215214 , ..., -0.4374012 ,
        -0.04366758, -0.13911277]], dtype=float32)>

In [ ]:
# Когда я скачиваю LaBSE-V2 encoder вручную с сайта, то
# при попытке чтения выдает ошибку KeyError: 'name' 
# Обходной путь:
# Этот скрипт скачивает LaBSE-V2 encoder как hub.KerasLayer и создает
# на его основе простую tf.keras.Model чтобы его обернуть и затем сохранить в файл, я
# был вынужден использовать Model поскольку нет встроенной функции для сохранения самого hub.KerasLayer 
#-------------------------------------------------------------------------------------------------------#

# import tensorflow_hub as hub
# LaBSEV2_encoder = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2", trainable=False)

# см. TF Functional API

# inp1 = tf.keras.Input(shape=(None,), dtype=tf.dtypes.int32, name='input_mask')
# inp2 = tf.keras.Input(shape=(None,), dtype=tf.dtypes.int32, name='input_word_ids')
# inp3 = tf.keras.Input(shape=(None,), dtype=tf.dtypes.int32, name='input_type_ids')
# 
# LaBSEV2_encoder({'input_mask': inp1, 'input_word_ids': inp2, 'input_type_ids': inp3}, False)
# model = tf.keras.Model(inputs=[inp1, inp2, inp3], outputs=LaBSEV2_encoder.output)
# model.save(proj_path + "stmodels\\LaBSE\\models\\LaBSEV2\\model\\")

Векторизация предложений и запись в соответствующую директорию

In [12]:
def myGetBatchEmbeddings(dfsentences_batch, encoderr, preprocessorr = None):
    embeddings = tf.constant([])
    if encoderr.enctype == SentenceEncodersTypes.SBERT:
        sentences = list(dfsentences_batch.values)
        embeddings = tf.constant(encoderr.encoder.encode(sentences))   # hahaha
            
    if encoderr.enctype == SentenceEncodersTypes.CMLM or encoderr.enctype == SentenceEncodersTypes.LaBSE:        
        if preprocessorr == None:
            raise KeyError("Preprocessor is required for encoder type : " + encoderr.enctype.value + ", please pass parameter: preprocessorr = <class: SentencePreprocessor>.")
        else:
            sentences = tf.constant(dfsentences_batch.values)
            embeddings = encoderr.encoder(preprocessorr.preprocessor(tf.constant(sentences)))["default"]
            
    return embeddings.numpy()

In [13]:
def mySaveEmbeddings(splittedsentences, encoder, preprocessor = None, usebatches = True, batch_size = 20):
    
    if usebatches == True:
        prep_modelname = ''
        if preprocessor != None:
            prep_modelname = preprocessor.modelname 
        
        embedd_dir = proj_path + dataset_path + "embeddings\\" + splittedsentences.method.value + '\\' + encoder.modelname + prep_modelname + '\\'
        batches_dir = embedd_dir + "batches\\"
        
                   
        embedd_name = dataset_name + '_' + splittedsentences.method.value + '_' + encoder.modelname + prep_modelname
        
        if os.path.isdir(batches_dir) == False:
            print("Creating directory: " +  batches_dir)
            os.makedirs(batches_dir)
        else:
            print("Clearing directory before loading: " + batches_dir)
            for filename in os.listdir(batches_dir):
                temp = batches_dir + filename
                if os.path.isfile(temp) or os.path.islink(temp):
                        os.unlink(temp)
            print("Done")
        
        ind_from = 0
        ind_to = 0
        l = splittedsentences.sentences.shape[0]

        print("Loading batches to: " + batches_dir)
        batchh_id = 0
        padzeroleft_batchhid = int(math.log10(int(l/batch_size + 1))) + 1
        while ind_to != l:
            ind_to = ind_from + batch_size
            if ind_to > l:
                ind_to = l

            batchh = splittedsentences.sentences.loc[splittedsentences.sentences.index[ind_from : ind_to], "CommentSentence"]
            batch_name = embedd_name + '_' + "batchid" + str(batchh_id).zfill(padzeroleft_batchhid) + '_' + str(ind_from) + '_' + str(ind_to) + ".npy"
            ind_from = ind_to
            batchh_id = batchh_id + 1
            
            batchh_embedd = myGetBatchEmbeddings(batchh, encoder, preprocessorr=preprocessor)
            np.save(batches_dir + batch_name, batchh_embedd)
            print("Done: " + str(ind_from) + " out of " + str(l) + "  " + str(int(ind_from/l*100)) + '%')
        print("Done")
        
        print("Reading and concatenating batches: ")
        arrs_to_concat = []
        for filename in os.listdir(batches_dir):
            temp = batches_dir + filename
            arrs_to_concat.append(np.load(temp))
        
        res = np.concatenate(arrs_to_concat, axis = 0)
        print("Done")
              
        np.save(embedd_dir + embedd_name + ".npy",res)
        print("Result saved as: " + embedd_dir + embedd_name + ".npy")
        
    else:
        raise KeyError("usebatches = False NOT IMPLEMENTED" )

In [26]:
mySaveEmbeddings(commentsNV,RuBERT180M,preprocessor=None,usebatches=True,batch_size=20)
mySaveEmbeddings(commentsNV,LaBSEV2,preprocessor=MultilingualV2,usebatches=True,batch_size=20)
mySaveEmbeddings(commentsNV,SentEvalV1,preprocessor=ENuncasedV3,usebatches=True,batch_size=20)


Creating directory: C:\Users\VG\Saved Games\Desktop\dippractice\data\politics\ru\putinspeech\embeddings\naive\RuBERT180M\batches\
Loading batches to: C:\Users\VG\Saved Games\Desktop\dippractice\data\politics\ru\putinspeech\embeddings\naive\RuBERT180M\batches\
Done: 20 out of 28058  0%
Done: 40 out of 28058  0%
Done: 60 out of 28058  0%
Done: 80 out of 28058  0%
Done: 100 out of 28058  0%
Done: 120 out of 28058  0%
Done: 140 out of 28058  0%
Done: 160 out of 28058  0%
Done: 180 out of 28058  0%
Done: 200 out of 28058  0%
Done: 220 out of 28058  0%
Done: 240 out of 28058  0%
Done: 260 out of 28058  0%
Done: 280 out of 28058  0%
Done: 300 out of 28058  1%
Done: 320 out of 28058  1%
Done: 340 out of 28058  1%
Done: 360 out of 28058  1%
Done: 380 out of 28058  1%
Done: 400 out of 28058  1%
Done: 420 out of 28058  1%
Done: 440 out of 28058  1%
Done: 460 out of 28058  1%
Done: 480 out of 28058  1%
Done: 500 out of 28058  1%
Done: 520 out of 28058  1%
Done: 540 out of 28058  1%
Done: 560 out o

Done: 5640 out of 28058  20%
Done: 5660 out of 28058  20%
Done: 5680 out of 28058  20%
Done: 5700 out of 28058  20%
Done: 5720 out of 28058  20%
Done: 5740 out of 28058  20%
Done: 5760 out of 28058  20%
Done: 5780 out of 28058  20%
Done: 5800 out of 28058  20%
Done: 5820 out of 28058  20%
Done: 5840 out of 28058  20%
Done: 5860 out of 28058  20%
Done: 5880 out of 28058  20%
Done: 5900 out of 28058  21%
Done: 5920 out of 28058  21%
Done: 5940 out of 28058  21%
Done: 5960 out of 28058  21%
Done: 5980 out of 28058  21%
Done: 6000 out of 28058  21%
Done: 6020 out of 28058  21%
Done: 6040 out of 28058  21%
Done: 6060 out of 28058  21%
Done: 6080 out of 28058  21%
Done: 6100 out of 28058  21%
Done: 6120 out of 28058  21%
Done: 6140 out of 28058  21%
Done: 6160 out of 28058  21%
Done: 6180 out of 28058  22%
Done: 6200 out of 28058  22%
Done: 6220 out of 28058  22%
Done: 6240 out of 28058  22%
Done: 6260 out of 28058  22%
Done: 6280 out of 28058  22%
Done: 6300 out of 28058  22%
Done: 6320 out

Done: 11260 out of 28058  40%
Done: 11280 out of 28058  40%
Done: 11300 out of 28058  40%
Done: 11320 out of 28058  40%
Done: 11340 out of 28058  40%
Done: 11360 out of 28058  40%
Done: 11380 out of 28058  40%
Done: 11400 out of 28058  40%
Done: 11420 out of 28058  40%
Done: 11440 out of 28058  40%
Done: 11460 out of 28058  40%
Done: 11480 out of 28058  40%
Done: 11500 out of 28058  40%
Done: 11520 out of 28058  41%
Done: 11540 out of 28058  41%
Done: 11560 out of 28058  41%
Done: 11580 out of 28058  41%
Done: 11600 out of 28058  41%
Done: 11620 out of 28058  41%
Done: 11640 out of 28058  41%
Done: 11660 out of 28058  41%
Done: 11680 out of 28058  41%
Done: 11700 out of 28058  41%
Done: 11720 out of 28058  41%
Done: 11740 out of 28058  41%
Done: 11760 out of 28058  41%
Done: 11780 out of 28058  41%
Done: 11800 out of 28058  42%
Done: 11820 out of 28058  42%
Done: 11840 out of 28058  42%
Done: 11860 out of 28058  42%
Done: 11880 out of 28058  42%
Done: 11900 out of 28058  42%
Done: 1192

Done: 16740 out of 28058  59%
Done: 16760 out of 28058  59%
Done: 16780 out of 28058  59%
Done: 16800 out of 28058  59%
Done: 16820 out of 28058  59%
Done: 16840 out of 28058  60%
Done: 16860 out of 28058  60%
Done: 16880 out of 28058  60%
Done: 16900 out of 28058  60%
Done: 16920 out of 28058  60%
Done: 16940 out of 28058  60%
Done: 16960 out of 28058  60%
Done: 16980 out of 28058  60%
Done: 17000 out of 28058  60%
Done: 17020 out of 28058  60%
Done: 17040 out of 28058  60%
Done: 17060 out of 28058  60%
Done: 17080 out of 28058  60%
Done: 17100 out of 28058  60%
Done: 17120 out of 28058  61%
Done: 17140 out of 28058  61%
Done: 17160 out of 28058  61%
Done: 17180 out of 28058  61%
Done: 17200 out of 28058  61%
Done: 17220 out of 28058  61%
Done: 17240 out of 28058  61%
Done: 17260 out of 28058  61%
Done: 17280 out of 28058  61%
Done: 17300 out of 28058  61%
Done: 17320 out of 28058  61%
Done: 17340 out of 28058  61%
Done: 17360 out of 28058  61%
Done: 17380 out of 28058  61%
Done: 1740

Done: 22220 out of 28058  79%
Done: 22240 out of 28058  79%
Done: 22260 out of 28058  79%
Done: 22280 out of 28058  79%
Done: 22300 out of 28058  79%
Done: 22320 out of 28058  79%
Done: 22340 out of 28058  79%
Done: 22360 out of 28058  79%
Done: 22380 out of 28058  79%
Done: 22400 out of 28058  79%
Done: 22420 out of 28058  79%
Done: 22440 out of 28058  79%
Done: 22460 out of 28058  80%
Done: 22480 out of 28058  80%
Done: 22500 out of 28058  80%
Done: 22520 out of 28058  80%
Done: 22540 out of 28058  80%
Done: 22560 out of 28058  80%
Done: 22580 out of 28058  80%
Done: 22600 out of 28058  80%
Done: 22620 out of 28058  80%
Done: 22640 out of 28058  80%
Done: 22660 out of 28058  80%
Done: 22680 out of 28058  80%
Done: 22700 out of 28058  80%
Done: 22720 out of 28058  80%
Done: 22740 out of 28058  81%
Done: 22760 out of 28058  81%
Done: 22780 out of 28058  81%
Done: 22800 out of 28058  81%
Done: 22820 out of 28058  81%
Done: 22840 out of 28058  81%
Done: 22860 out of 28058  81%
Done: 2288

...

Done: 4520 out of 28058  16%
Done: 4540 out of 28058  16%
Done: 4560 out of 28058  16%
Done: 4580 out of 28058  16%
Done: 4600 out of 28058  16%
Done: 4620 out of 28058  16%
Done: 4640 out of 28058  16%
Done: 4660 out of 28058  16%
Done: 4680 out of 28058  16%
Done: 4700 out of 28058  16%
Done: 4720 out of 28058  16%
Done: 4740 out of 28058  16%
Done: 4760 out of 28058  16%
Done: 4780 out of 28058  17%
Done: 4800 out of 28058  17%
Done: 4820 out of 28058  17%
Done: 4840 out of 28058  17%
Done: 4860 out of 28058  17%
Done: 4880 out of 28058  17%
Done: 4900 out of 28058  17%
Done: 4920 out of 28058  17%
Done: 4940 out of 28058  17%
Done: 4960 out of 28058  17%
Done: 4980 out of 28058  17%
Done: 5000 out of 28058  17%
Done: 5020 out of 28058  17%
Done: 5040 out of 28058  17%
Done: 5060 out of 28058  18%
Done: 5080 out of 28058  18%
Done: 5100 out of 28058  18%
Done: 5120 out of 28058  18%
Done: 5140 out of 28058  18%
Done: 5160 out of 28058  18%
Done: 5180 out of 28058  18%
Done: 5200 out

Done: 10180 out of 28058  36%
Done: 10200 out of 28058  36%
Done: 10220 out of 28058  36%
Done: 10240 out of 28058  36%
Done: 10260 out of 28058  36%
Done: 10280 out of 28058  36%
Done: 10300 out of 28058  36%
Done: 10320 out of 28058  36%
Done: 10340 out of 28058  36%
Done: 10360 out of 28058  36%
Done: 10380 out of 28058  36%
Done: 10400 out of 28058  37%
Done: 10420 out of 28058  37%
Done: 10440 out of 28058  37%
Done: 10460 out of 28058  37%
Done: 10480 out of 28058  37%
Done: 10500 out of 28058  37%
Done: 10520 out of 28058  37%
Done: 10540 out of 28058  37%
Done: 10560 out of 28058  37%
Done: 10580 out of 28058  37%
Done: 10600 out of 28058  37%
Done: 10620 out of 28058  37%
Done: 10640 out of 28058  37%
Done: 10660 out of 28058  37%
Done: 10680 out of 28058  38%
Done: 10700 out of 28058  38%
Done: 10720 out of 28058  38%
Done: 10740 out of 28058  38%
Done: 10760 out of 28058  38%
Done: 10780 out of 28058  38%
Done: 10800 out of 28058  38%
Done: 10820 out of 28058  38%
Done: 1084

Done: 15660 out of 28058  55%
Done: 15680 out of 28058  55%
Done: 15700 out of 28058  55%
Done: 15720 out of 28058  56%
Done: 15740 out of 28058  56%
Done: 15760 out of 28058  56%
Done: 15780 out of 28058  56%
Done: 15800 out of 28058  56%
Done: 15820 out of 28058  56%
Done: 15840 out of 28058  56%
Done: 15860 out of 28058  56%
Done: 15880 out of 28058  56%
Done: 15900 out of 28058  56%
Done: 15920 out of 28058  56%
Done: 15940 out of 28058  56%
Done: 15960 out of 28058  56%
Done: 15980 out of 28058  56%
Done: 16000 out of 28058  57%
Done: 16020 out of 28058  57%
Done: 16040 out of 28058  57%
Done: 16060 out of 28058  57%
Done: 16080 out of 28058  57%
Done: 16100 out of 28058  57%
Done: 16120 out of 28058  57%
Done: 16140 out of 28058  57%
Done: 16160 out of 28058  57%
Done: 16180 out of 28058  57%
Done: 16200 out of 28058  57%
Done: 16220 out of 28058  57%
Done: 16240 out of 28058  57%
Done: 16260 out of 28058  57%
Done: 16280 out of 28058  58%
Done: 16300 out of 28058  58%
Done: 1632

Done: 21140 out of 28058  75%
Done: 21160 out of 28058  75%
Done: 21180 out of 28058  75%
Done: 21200 out of 28058  75%
Done: 21220 out of 28058  75%
Done: 21240 out of 28058  75%
Done: 21260 out of 28058  75%
Done: 21280 out of 28058  75%
Done: 21300 out of 28058  75%
Done: 21320 out of 28058  75%
Done: 21340 out of 28058  76%
Done: 21360 out of 28058  76%
Done: 21380 out of 28058  76%
Done: 21400 out of 28058  76%
Done: 21420 out of 28058  76%
Done: 21440 out of 28058  76%
Done: 21460 out of 28058  76%
Done: 21480 out of 28058  76%
Done: 21500 out of 28058  76%
Done: 21520 out of 28058  76%
Done: 21540 out of 28058  76%
Done: 21560 out of 28058  76%
Done: 21580 out of 28058  76%
Done: 21600 out of 28058  76%
Done: 21620 out of 28058  77%
Done: 21640 out of 28058  77%
Done: 21660 out of 28058  77%
Done: 21680 out of 28058  77%
Done: 21700 out of 28058  77%
Done: 21720 out of 28058  77%
Done: 21740 out of 28058  77%
Done: 21760 out of 28058  77%
Done: 21780 out of 28058  77%
Done: 2180

Done: 26620 out of 28058  94%
Done: 26640 out of 28058  94%
Done: 26660 out of 28058  95%
Done: 26680 out of 28058  95%
Done: 26700 out of 28058  95%
Done: 26720 out of 28058  95%
Done: 26740 out of 28058  95%
Done: 26760 out of 28058  95%
Done: 26780 out of 28058  95%
Done: 26800 out of 28058  95%
Done: 26820 out of 28058  95%
Done: 26840 out of 28058  95%
Done: 26860 out of 28058  95%
Done: 26880 out of 28058  95%
Done: 26900 out of 28058  95%
Done: 26920 out of 28058  95%
Done: 26940 out of 28058  96%
Done: 26960 out of 28058  96%
Done: 26980 out of 28058  96%
Done: 27000 out of 28058  96%
Done: 27020 out of 28058  96%
Done: 27040 out of 28058  96%
Done: 27060 out of 28058  96%
Done: 27080 out of 28058  96%
Done: 27100 out of 28058  96%
Done: 27120 out of 28058  96%
Done: 27140 out of 28058  96%
Done: 27160 out of 28058  96%
Done: 27180 out of 28058  96%
Done: 27200 out of 28058  96%
Done: 27220 out of 28058  97%
Done: 27240 out of 28058  97%
Done: 27260 out of 28058  97%
Done: 2728

## Data Dimensionality Reducers

UMAP

In [ ]:
UMAP_reducer = umap.UMAP(a=None, angular_rp_forest=False, b=None,
     force_approximation_algorithm=False, init='spectral', learning_rate=1.0,
     local_connectivity=1.0, low_memory=False, metric='euclidean',
     metric_kwds=None, min_dist=0.1, n_components=2, n_epochs=None,
     n_neighbors=15, negative_sample_rate=5, output_metric='euclidean',
     output_metric_kwds=None, random_state=42, repulsion_strength=1.0,
     set_op_mix_ratio=1.0, spread=1.0, target_metric='categorical',
     target_metric_kwds=None, target_n_neighbors=-1, target_weight=0.5,
     transform_queue_size=4.0, transform_seed=42, unique=False, verbose=False)



PCA

In [ ]:
PCA_reducer = decomposition.PCA(n_components=None, copy=True, whiten=False, svd_solver='auto', tol=0.0, iterated_power='auto', random_state=42)